This will the file I use to do scratch/one time use stuff.

In [2]:
from datasets import Dataset
import pandas as pd
import re
from dateutil import parser
from pathlib import Path
from string import punctuation
import math

/home/dck5549/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#taken from 01_parse.ipynb

def check_trans(word_list, messages):
    all_words = {}
    counter = {} # includes actual words from conversation
    dict_counter = {} # includes words from dictionary
    
    for message in messages:
        content = message.split(" ")
        for word in content:
            word = word.strip(punctuation).lower()
            if len(word)>1:
                if word in all_words:
                    all_words[word] += 1
                elif word.isalpha() == True:
                    all_words[word] = 1
                else:
                    if word[0].isnumeric() == False:
                        for symbol in punctuation:
                            if symbol in word:
                                split_word = word.split(symbol)
                                for section in split_word:
                                    if len(section) > 1:
                                        if word in all_words:
                                            all_words[word] += 1
                                        else:
                                            if word.isalpha() == True:
                                                all_words[word] = 1
                for check in word_list:
                    find = re.match(check, word)
                    if find != None:
                        if check[-1] != "*":
                            if len(word) > find.span()[1]:
                                continue
                        else:
                            if len(word) < len(check):
                                continue
                        if word not in counter:
                            counter[word] = 1
                        else:
                            counter[word] += 1
                        if check not in dict_counter:
                            dict_counter[check] = 1
                        else:
                            dict_counter[check] += 1
    return counter, dict_counter, all_words

In [5]:
#taken from 01_parse.ipynb
# Helper functions to parse transcripts, adapted from megan's code
# Input: path to directory full of transcripts
# Output: a DataFrame of all the utterances in the transcript and their speaker
def parse_trans(path):

   tran_path = Path(path)
   tran_list = list(tran_path.glob('*.txt'))
   transcripts = {}
   time = 0
   for filepath in tran_list:
      name = Path(filepath).stem
      with open(filepath,'r',encoding='utf-8') as my_file:
         data = my_file.readlines()
         clean = []
         for line in data:
            if ":" in line and line[:1].isnumeric() == False:
               clean.append(line.strip() + "&&" +  str(time))
            elif ":" in line and line[:1].isnumeric() == True:
               time +=1
      transcripts[name] = clean


   last_time = 0
   trans_df = pd.DataFrame()
   for session in transcripts:
      s_df = pd.DataFrame.from_dict(transcripts[session])
      def trans_proccess(c): 

         global g_time
         x=c[0].strip()
         # print(x)
         try:
            speaker = re.search(r"^[^:]*:\s*", x).group()[:-2]
         except:
            speaker = ""
         try:
            content = re.search(r":(.*)&&", x).group()[1:-2]
         except:
            content = ""
         try:
            time = int(re.search(r"&&.*", x).group()[2:])
            g_time = time
         except:
            time = -1

         if(speaker == "" and content == ""):
            speaker = None
            content = None

         row = pd.Series(dtype='float64')

         row['speaker'] = speaker
         row['content'] = content
         row['block'] = time-last_time 
         row['session'] = session
         return row
      trans_df = pd.concat([trans_df, s_df.apply(lambda x: trans_proccess(x), axis=1)])
      last_time = g_time
   trans_df = trans_df.dropna()
   trans_df = trans_df[trans_df['block'] >= 0].reset_index(drop=True)
   trans_df['block'] = (trans_df['block'] / trans_df['block'].max() * num_blocks).apply(math.floor)
   trans_df['session'] = trans_df['session'].astype('int')
   return trans_df

In [15]:
#Parsing NEK21_MAG/13.txt so it can be hand labeled
folder = "temp_folder"
num_blocks = 3
mag_13 = parse_trans(folder)

In [18]:
content = mag_13['content']

In [20]:
# Save Data
mag_13.to_csv("temp_folder/NEK21_MAG13.csv")

Below I do some testing of the already trained models. Code is from 02_classify

In [3]:
# %%
from datasets import Dataset
import pandas as pd
from transformers import BertForSequenceClassification, AutoTokenizer
import numpy as np


In [22]:
# %%
#load model
model = BertForSequenceClassification.from_pretrained("../models/model_mrda_v2_t1.model")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

labels = ["statement", "disruption", "backchannel", "follow-me", "question"]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}


(…)cased/resolve/main/tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 222kB/s]
(…)rt-base-uncased/resolve/main/config.json: 100%|██████████| 570/570 [00:00<00:00, 2.44MB/s]
(…)bert-base-uncased/resolve/main/vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 67.0MB/s]
(…)base-uncased/resolve/main/tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 144MB/s]


In [29]:

# Preproccesses data before classifcation
def preproccess(samples):
    encoding = tokenizer.encode_plus(samples['content'], add_special_tokens = True,
                        max_length = 32,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                        truncation=True,
                        padding="max_length")
  
    samples['input_ids'] = encoding['input_ids']
    samples['token_type_ids'] = encoding['token_type_ids']
    samples['attention_mask'] = encoding['attention_mask']
    return samples


# %%
# Uses trained model to classify
def classify(samples):
    out = model(samples['input_ids'], token_type_ids=samples['token_type_ids'], attention_mask=samples['attention_mask'])
    logits = out.logits.detach().cpu().numpy()

    samples['logits'] = logits[0]
    samples['labels_h'] = labels[logits.argmax()]
    samples['labels'] = logits.argmax()
    return samples


In [30]:
#Load in chat & transcript data
NEK21_df = pd.read_csv('temp_folder/NEK21_MAG13.csv')
NEK21_df = NEK21_df.dropna()

NEK21 = Dataset.from_pandas(NEK21_df)



NEK21 = NEK21.map(preproccess)
NEK21.set_format('torch')
NEK21 = NEK21.map(classify)

NEK21.to_csv('temp_folder/NEK21_13_results.csv')


Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  4.62ba/s]


445840

In [4]:
#load in results from ^, and the hand-labeled results 
labels = ["s", "d", "b", "f", "q"]
label2id = {label:idx for idx, label in enumerate(labels)}

preds = pd.read_csv("temp_folder/NEK21_13_results.csv")
hand_labled = pd.read_csv("fewshot_labels/NEK21_MAG13.csv")

In [5]:
from sklearn.preprocessing import OneHotEncoder

# preproccess BERT machine labeled labels by putting them into one-hot format
onehot_enc = OneHotEncoder(sparse=False)
bert_labels_raw = preds['labels'][0:].to_numpy().reshape(preds['labels'].size,1)
bert_labels = onehot_enc.fit_transform(bert_labels_raw)

/home/dck5549/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:

# transform hand labels to one-hot format
h1_raw = hand_labled
h1_raw = h1_raw.replace({'labels_h': label2id})['labels_h'].to_numpy().reshape([-1,1])
h1_labels = onehot_enc.fit_transform(h1_raw)

/home/dck5549/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
#model never predicts disruption?? This is also true in the 03_bert_validation
h,hc = np.unique(h1_raw, return_counts=True)
b,bc = np.unique(bert_labels_raw, return_counts=True)

print("Distribution of Hand Labels:")
for i, val in enumerate(h):
    print(f'{labels[val]}: {hc[i]}')

print("Distribution of Model Labels:")
for i, val in enumerate(b):
    print(f'{labels[val]}: {bc[i]}')


Distribution of Hand Labels:
s: 354
d: 75
b: 50
f: 4
q: 142
Distribution of Model Labels:
s: 451
b: 12
f: 2
q: 160


In [13]:
correct = 0
s_correct = 0
q_correct = 0
d_correct = 0
b_correct = 0
f_correct = 0


for i,l in enumerate(h1_raw):

    if l[0] == bert_labels_raw[i][0]:
        correct += 1

        t = labels[l[0]]
        if t == "s":
            s_correct += 1
        elif t == "q":
            q_correct += 1
        elif t == "d":
            d_correct += 1
        elif t == "f":
            f_correct += 1
        else:
            b_correct += 1
        

print(f'Total Accuracy: {correct/len(h1_raw)}%')
print(f'Statement Accuracy: {s_correct/hc[0]}%')
print(f'Disruption Accuracy: {d_correct/hc[1]}%')
print(f'Backchannel Accuracy: {b_correct/hc[2]}%')
print(f'Follow-me Accuracy: {f_correct/hc[3]}%')
print(f'Question Accuracy: {q_correct/hc[4]}%')

Total Accuracy: 0.7648%
Statement Accuracy: 0.943502824858757%
Disruption Accuracy: 0.0%
Backchannel Accuracy: 0.12%
Follow-me Accuracy: 0.0%
Question Accuracy: 0.971830985915493%
